# Text Detection
## 1. Định hướng

Thách thức đặt ra là làm sao để xác định các đối tượng vừa nhận diẹn được có chứa văn bản để phân loại chúng. Để giải quyết vấn đề này ta cần một tập dữ liệu huấn luyện lý tưởng gồm 50% số hình có chứa văn bản và 50% còn lại không chứa. Để tạo ra tập dữ liệu này ta cần kết hợp 2 tập dữ liệu:  
1. 50 000 ảnh được lấy từ 78 903 hình ở tập dữ liệu [74K Chars dataset](http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/).Tập này gồm những hình ảnh có văn bản và được dán nhãn là 1
2. 50 000 ảnh được lấy từ tập [CIFAR-10 dataset on Kaggle](https://www.kaggle.com/c/cifar-10/data). Những hình ảnh trong tập này không có văn bản và được dán nhãn là 0  
  
Tập hoàn chỉnh gồm 100 000 ảnh, được dán nhãn đúng và trộn lại theo 1 thứ tự ngẫu nhiên. Do vậy, ta cần một mô hình phân lớp nhị phân.  

Mô hình này gồm 2 bước:  
1. **Trích xuất các đặc trưng:** Bước này sẽ tính toán [Histogram of Gradient](http://scikit-image.org/docs/dev/auto_examples/plot_hog.html) (HOG) của ảnh. Kỹ thuật ở đây dựa vào việc ta có thể mô tả thể hiện và hình dáng của các đối tượng trong ảnh bằng sự phân bố cường độ các gradient (gradient là sự thay đổi hướng của cường độ hoặc màu của ảnh). Cách này thường được sử dụng trong việc nhận diện đối tượng vì nó có thể tìm ra các đường bao của hình dáng một cách tương đối dễ dàng. Khi thực hiện xong bước này, ta được một tập các điểm dữ liệu đặc trưng cần thiết cho việc phân loại.  
2. **Phân loại**: Bước này sử dụng phương pháp [Support Vector Machines with a Linear Kernel](http://scikit-learn.org/stable/modules/svm.html) (SVM)

## 2. Thực hiện
Nhóm đã xây dựng các lớp sau cho việc train dữ liệu và đánh giá mô hình:
1. Lớp **CifarKaggle** dùng cho việc đọc và xử lý dữ liệu không chứa text trong tập **CIFAR**. Dữ liệu được đọc từ file **cifar-cofig.py** - được lưu dạng dictionary. Dictionary gồm các key như sau: 
     - **'verbose'**: True - ghi các kết quả từng bước ra màn hình, False thì ngược lại  
     - **'from_pickle'**: True - đọc file dữ liệu từ pickle_data. False thì load ảnh từ folder và ghi vào file pickle  
     - **'pickle_data'**: File pickle lưu dữ liệu ảnh đã load, tiết kiệm thời gian  
     - **'folder'**: Folder chứa ảnh cần load lên, cũng là thư mục chứa pickle_data   
     - **'img_size'**: Kích thước ảnh cần resize  
    
2. Lớp **OcrData** dùng cho việc tạo tập dữ liệu 100000 ảnh, train dữ liệu và đánh giá các mô hình. Có các hàm chính sau:  
  - **merge_with_cifar**: lấy dữ liệu 50000 ảnh chứa kí tự từ file **text-config.py** (tập 74K Chars) trộn với 50000 ảnh không chứa kí tự từ file **cifar-config.py** (tập CIFAR) để tạo thành tập train  
  - **perform_grid_search_cv**: Dùng Grid Search Cross Validation để đánh giá các tham số khác nhau cho thuật toán SVM + HOG để tìm ra được bộ tham số có score tốt nhất.  
  - **generate_best_hog_model**: Khi đã có bộ tham số thì ta dựng lại mô hình trên toàn bộ tập train 100000  
  - **evaluate**: Đánh giá và vẽ biểu đồ biểu thị độ chính xác của mô hình trên toàn bộ tập train 100000    
  
  File **text-cofig.py** và **ocr-config.py** đều có chung cấu trúc như sau:    
  - **'from_pickle', 'pickle_data', 'folder_data', 'verbose', 'img_size'**: tương tự như file  **cifar-cofig.py**  
  - **'folder_labels'**: Thư mục lưu nhãn của tập dữ liệu (nếu không load từ file pickle)  
  - **'limit'**: giới hạn số ảnh cần load  
  - **'automatic_split'**: True - tự động chia tập dữ liệu thành train và test, False - ngược lại  
  - **'plot_evaluation'**: True - biểu diễn dữ liệu, False - ngược lại  
  - **'percentage_of_test_set'**: Tỉ lệ dữ liệu test - để chia dữ liệu 
  
Bổ sung thêm phương thức **select_text_among_candidates** cho lớp **ImageData**, để lọc các đối tượng chứa text dựa vào mô hình đã dựng được  
## 3. Kết quả
Sau khi đánh giá các mô hình khác nhau bằng phương pháp **Grid Search Cross Validation**, ta được kết quả như sau:  
Sau khi chạy 324 bộ tham số cho mô hình SVM-HOG, mỗi bộ tham số chạy 3 lần, ta được tham số có score validation cao nhất là

Sau khi dựng lại mô hình trên toàn bộ tập 100000 ảnh, ta được kết quả như sau về độ chính xác:  
![generate_best_hog_model](img/generate_best_hog_model.jpg)  
Sau khi đánh giá mô hình, ta được kết quả như sau:  
![evaluate](img/evaluate.jpg)  
Biểu đồ confusion_matrix biểu diễn độ chính xác của mô hình:  
![figure](img/figure.png)  
  
Kết quả sau khi lọc các đối tượng chứa text, lọc được 66/203 đối tượng có chứa text:  
![select_text](img/select_text.jpg)  
Kết quả lọc được:  
![text_objects](img/text_objects.jpg)  